<a href="https://colab.research.google.com/github/OrLehmann/Projects/blob/main/classification_reviews_embedding_and_connecting_BigQuery_version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import json
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import openai
import numpy as np




In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your API key file
api_key_file = '/content/drive/MyDrive/openai_api_key.txt'  # Adjust the path if necessary

# Read the API key from the file
with open(api_key_file, 'r') as file:
    api_key = file.read().strip()

# Set the environment variable
os.environ['OPENAI_API_KEY'] = api_key

# Optional: Verify that the environment variable is set
print("API Key Loaded:", os.environ.get('OPENAI_API_KEY') is not None)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
API Key Loaded: True


In [ ]:
# Define the path to your BigQuery service account key file
service_account_path = '/content/drive/MyDrive/bigquery_service_account.json'  # Update with the correct path

# Authenticate with BigQuery
credentials = service_account.Credentials.from_service_account_file(service_account_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)


In [ ]:
# Define dataset and table information
PROJECT_ID = credentials.project_id
DATASET_ID = "hotel_feedback"
TABLE_ID = "classified_reviews"

In [ ]:
# Load Excel file
file_path = "/content/drive/My Drive/Caesar eilat unclassified reviews.csv"
df = pd.read_csv(file_path, usecols=["ID","feedback", "Site", "Month", "Rating", "Pos=1/Neg=0"])
df.rename(columns={"Pos=1/Neg=0": "Pos_Neg"}, inplace=True)
df["Month"] = df["Month"].astype(str)  # Convert month to string




In [ ]:
# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Department categories
departments = [
    "Front Desk", "Food and Beverages", "Executive Chef", "Food and Beverage Auditor",
    "Housekeeping", "Security", "Accountant", "Entertainment and Activities",
    "Procurement", "Pool", "Reservations", "Groups and Public Relations",
    "Human Resources and Stewards", "Maintenance", "Guest Service Center",
    "Evening Manager", "General Manager", "Feedback with names"
]

In [ ]:
# Compute embeddings for department names using "text-embedding-3-large"

def get_embeddings(texts):
    response = openai.embeddings.create(
        input=texts,  # Send a batch of texts
        model="text-embedding-3-large"
    )
    return [np.array(e.embedding) for e in response.data]

# Compute department embeddings in a batch
department_embeddings = np.array(get_embeddings(departments))

# Function to classify reviews using embeddings
def classify_reviews_with_embedding(reviews):
    """Classifies multiple reviews using embedding similarity with text-embedding-3-large."""
    review_embeddings = get_embeddings(reviews)  # Get embeddings in batch
    classified_results = []
    for review_embedding in review_embeddings:
        similarities = np.dot(department_embeddings, review_embedding)  # Cosine similarity approximation
        top_indices = np.where(similarities > 0.3)[0]  # Thresholding instead of fixed top-2
        if len(top_indices) == 0:
            top_indices = [np.argmax(similarities)]  # Ensure at least 1 category
        classified_results.append([departments[i] for i in top_indices])
    return classified_results




In [ ]:
# Classify reviews in batch
batch_size = 35  # Adjust batch size as needed
reviews_list = df["feedback"].tolist()
categories_list = []

for i in range(0, len(reviews_list), batch_size):
    batch_reviews = reviews_list[i:i+batch_size]
    batch_categories = classify_reviews_with_embedding(batch_reviews)
    categories_list.extend(batch_categories)

df["categories"] = categories_list

# Ensure 'categories' is always a list (important for BigQuery)
df["categories"] = df["categories"].apply(lambda x: x if isinstance(x, list) else [x])


# Convert dataframe to JSON records
json_data = df.to_dict(orient="records")
output_json_path = "classified_reviews.json"
# Write NDJSON format (each JSON object on a new line)
with open(output_json_path, "w", encoding="utf-8") as f:
    for record in json_data:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")
print(f"✅ JSON file saved at: {output_json_path}")


✅ JSON file saved at: classified_reviews.json


In [ ]:
# Define BigQuery Schema
schema = [
    bigquery.SchemaField("ID", "STRING"),
    bigquery.SchemaField("feedback", "STRING"),
    bigquery.SchemaField("Site", "STRING"),
    bigquery.SchemaField("Month", "STRING"),
    bigquery.SchemaField("Rating", "INTEGER"),
    bigquery.SchemaField("Pos_Neg", "STRING"),
    bigquery.SchemaField("categories", "STRING", mode="REPEATED")
]

# Ensure dataset exists
dataset_ref = client.dataset(DATASET_ID)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"
try:
    client.create_dataset(dataset)
    print(f"Dataset {DATASET_ID} created.")
except Exception:
    print(f"Dataset {DATASET_ID} already exists.")

# Ensure table exists
table_ref = dataset_ref.table(TABLE_ID)
table = bigquery.Table(table_ref, schema=schema)
try:
    client.create_table(table)
    print(f"Table {TABLE_ID} created.")
except Exception:
    print(f"Table {TABLE_ID} already exists.")

Dataset hotel_feedback already exists.
Table classified_reviews already exists.


In [ ]:
# Load JSON data into BigQuery
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    autodetect=False
)

# Validate JSON file before uploading
with open(output_json_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            json.loads(line)  # Ensure each line is a valid JSON object
        except json.JSONDecodeError as e:
            print(f"Invalid JSON line: {line}")
            print(e)

# Upload JSON to BigQuery
with open(output_json_path, "rb") as json_file:
    job = client.load_table_from_file(json_file, table_ref, job_config=job_config)
    job.result()  # Wait for completion


print(f"✅ Data uploaded to BigQuery: {PROJECT_ID}.{DATASET_ID}.{TABLE_ID}")


✅ Data uploaded to BigQuery: gifted-proxy-452616-s7.hotel_feedback.classified_reviews


In [ ]:
import json

# Define the path for the new JSON file
classified_reviews_output_path = "/content/drive/MyDrive/classified_reviews_export.json"

# Convert dataframe to JSON records
classified_reviews_json = df.to_dict(orient="records")

# Write final classified reviews JSON
with open(classified_reviews_output_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print(f"✅ Classified reviews exported to: {classified_reviews_output_path}")

✅ Classified reviews exported to: /content/drive/MyDrive/classified_reviews_export.json
